# 목표 
이전 이미지 관련 실습에서는 사용자가 이미지를 업로드하면 하드 코딩 된 prompt와 함께 GPT로 보내 답변을 받는 챗봇을 구현했습니다. 이번에는 다음과 같이 기능을 확장하는 것이 과제의 목표입니다:

- 여러 이미지를 입력으로 받기
    - 기존에는 이미지 한 장만을 입력으로 받았다면 이번에는 다수의 사진을 입력 받을 수 있도록 만들어야 합니다.
    - Streamlit의 `st.file_uploader` [문서](https://docs.streamlit.io/develop/api-reference/widgets/st.file_uploader)를 확인하여 기존 코드에서 여러 장의 사진을 받을 수 있도록 구현해봅시다.
- 업로드 된 이미지들을 가지고 자유롭게 질의응답 할 수 있는 챗봇 구현
    - 기존과 다르게 하드코딩 된 prompt가 아닌, 사용자로부터 질문을 입력받아 GPT에게 넘겨주는 챗봇을 구현하셔야 합니다.
    - 그리고 사용자가 여러 번 질문을 입력해도 처음 주어진 이미지들로 답변할 수 있도록 구현하셔야 합니다(이 부분은 RAG 실습 코드를 참조).
    - GPT에게 여러 개의 사진을 보내주는 부분은 [API 문서](https://platform.openai.com/docs/guides/vision)를 확인하여 구현하시면 됩니다.
- 다음 이미지들과 질문에 대한 챗봇의 답변 생성
    - 다음 주어진 이미지들과 질문을 실제로 구현한 챗봇에게 주어졌을 때 어떤 답변이 생성되는지 영상을 녹화하셔야 합니다:
        - 이미지: 인터넷에서 강아지 사진과 고양이 사진 각각 1장씩 찾아 입력으로 쓰시면 됩니다.
        - 질문 1: 주어진 두 사진의 공통점이 뭐야?
        - 질문 2: 주어진 두 사진의 차이점이 뭐야?
    - 질문 1, 2를 차례로 입력하시면 됩니다. 즉, 챗봇과 질의응답이 두 차례 이루어져야 합니다.


#

# import 처리합니다

In [1]:
import streamlit as st
from PIL import Image
from io import BytesIO

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 이미지 업로드 InputView

- 여러 이미지를 입력받기

In [3]:
# OpenAI 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

# Streamlit 세션 초기화
if "messages" not in st.session_state:
    st.session_state.messages = []
if "uploaded_images" not in st.session_state:
    st.session_state.uploaded_images = []

# 이미지 업로드
uploaded_files = st.file_uploader(
    "최대 3개의 이미지를 업로드 해주세요",
    type=["jpg", "jpeg", "png"],
    accept_multiple_files=True,
)

if uploaded_files:
    for file in uploaded_files:
        image = Image.open(file)
        buffer = BytesIO()
        image.save(buffer, format="PNG")
        st.session_state.uploaded_images.append(
            {"name": file.name, "data": buffer.getvalue()}
        )
        st.image(image, caption=f"업로드된 이미지: {file.name}", use_column_width=True)

    st.success(f"{len(uploaded_files)}개의 이미지가 업로드되었습니다.")

# 사용자 대화 입력
prompt = st.chat_input("이미지에 대해 질문을 입력하세요 (예: 공통점은 무엇인가요?):")

if prompt:
    with st.chat_message("user"):
        st.markdown(prompt)
    st.session_state.messages.append({"role": "user", "content": prompt})

    # 이미지 설명 생성 및 모델 호출
    if len(st.session_state.uploaded_images) > 1:
        image_descriptions = "\n".join(
            [
                f"이미지 {i + 1}: {img['name']}"
                for i, img in enumerate(st.session_state.uploaded_images)
            ]
        )
        full_prompt = (
            f"다음은 사용자가 업로드한 이미지 목록입니다:\n{image_descriptions}\n\n"
            f"사용자의 질문: {prompt}\n\n"
            f"이미지의 공통점이나 차이점을 설명하고 질문에 답변해주세요."
        )
    else:
        full_prompt = f"사용자의 질문: {prompt}"

    with st.chat_message("assistant"):
        st.markdown("응답 생성 중...")

        # GPT 모델 호출
        result = model([HumanMessage(content=full_prompt)])
        response = result.content.strip()

        st.markdown(response)
        st.session_state.messages.append({"role": "assistant", "content": response})

# 업로드된 이미지 목록 및 대화 기록 표시
if st.session_state.uploaded_images:
    st.write("### 업로드된 이미지 목록")
    cols = st.columns(len(st.session_state.uploaded_images))
    for i, img in enumerate(st.session_state.uploaded_images):
        with cols[i]:
            st.image(
                Image.open(BytesIO(img["data"])),
                caption=f"이미지 {i + 1}: {img['name']}",
            )

2025-01-28 13:12:42.558 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-28 13:12:42.565 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-28 13:12:42.567 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-28 13:12:42.569 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-28 13:12:42.571 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-28 13:12:42.573 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-28 13:12:42.576 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-28 13:12:42.577 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar